In [1]:
import os

In [2]:
os.chdir('..')

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/dishan-nithi/credit-card-default-prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "dishan-nithi"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f9555631638b9db78b3221c99340c604de4fe75d"

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [16]:
from default_predictor.constants import *
from default_predictor.utils.common import read_yaml, create_directories, save_json

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBoost
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/dishan-nithi/credit-card-default-prediction.mlflow"
            
        )
        
        return model_evaluation_config

In [22]:
import os
import pandas as pd
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from urllib.parse import urlparse
import mlflow
from mlflow import sklearn, xgboost
import joblib

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        roc_auc = roc_auc_score(actual, pred)
        f1 = f1_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        accuracy = accuracy_score(actual, pred)
        return roc_auc, f1, precision, recall, accuracy
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            y_pred = model.predict(X_test)
            (roc_auc, f1, precision, recall, accuracy) = self.eval_metrics(y_test, y_pred)
            scores ={"roc_auc":roc_auc, "f1":f1, "precision":precision, "recall":recall, "accuracy":accuracy}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("roc_auc", roc_auc)
            mlflow.log_metric("f1", f1)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("accuracy", accuracy)
            
            if tracking_url_type_store != "file":
                mlflow.xgboost.log_model(model, "model", registered_model_name="XGBoostModel")

            else:
                mlflow.xgboost.log_model(model, "model")


In [28]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-13 15:26:41,570: INFO: common: YAML file config\config.yaml loaded successfuly]
[2024-12-13 15:26:41,572: INFO: common: YAML file params.yaml loaded successfuly]
[2024-12-13 15:26:41,583: INFO: common: YAML file schema.yaml loaded successfuly]
[2024-12-13 15:26:41,585: INFO: common: created directory at artifacts]
[2024-12-13 15:26:41,586: INFO: common: created directory at artifacts/model_evaluation]
[2024-12-13 15:26:42,583: INFO: common: JSON file saved at artifacts\model_evaluation\metrics.json]


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:26:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
Registered model 'XGBoostModel' already exists. Creating a new version of this model...
2024/12/13 15:26:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGBoostModel, version 5
Created version '5' of model 'XGBoostModel'.


A
https
